In [ ]:
import os

import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

import pandas as pd

In [ ]:
sf_account_id = 'CU71622'
sf_password = '123456Carne'
sf_login = 'dimeno157'

database_name = 'TEST_DATABASE'
schema_name = 'DEV'
table_name = 'ACCOUNTS'

In [ ]:
df = pd.read_csv('../data/accounts.csv')
df.columns = [x.upper() for x in df.columns]
df.head()

In [ ]:
# Make connection to snowflake
cnx = snowflake.connector.connect(
    user = sf_login,
    password = sf_password,
    account = sf_account_id,
    database = database_name,
    schema = schema_name,
    warehouse = 'TEST_WAREHOUSE',
    role = 'ACCOUNTADMIN'
)

cs = cnx.cursor()

table_path = f'{database_name}.{schema_name}.{table_name}'

In [ ]:
# Ceate a table
create_table = f'''
    CREATE OR REPLACE TABLE {table_path} (
        account_number varchar(99999),
        aba int,
        bic varchar(99999),
        opened varchar(99999),
        balance int
    ) 
'''
cs.execute(create_table)

In [ ]:
# Write on table
write_pandas(cnx, df, table_name=table_name)

In [ ]:
# SQL query table
cs.execute(f'''
    SELECT ACCOUNT_NUMBER, OPENED 
    FROM {table_path} 
    LIMIT 10;
''')
rows = cs.fetchall()
for row in rows:
    print(row)

In [ ]:
# Dataframe from snowflake
sql = '''SELECT * FROM ACCOUNTS;'''
cs.execute(sql)

df_from_df = cs.fetch_pandas_all()
df_from_df.head()

In [ ]:
cs.close()
cnx.close()